In [ ]:
#import libraries
!pip install kaggle
import kaggle

In [9]:
#download dataset using kaggle api
!kaggle datasets download -d ankitbansal06/retail-orders

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
retail-orders.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
#extract file from zip file

import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file

In [27]:
#read data from the file and handle null values

import pandas as pd
df = pd.read_csv('orders.csv', na_values = ['unknown', 'Not Available'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [43]:
#rename columns names ..make them lower case and replace space with underscore

#df.rename(columns={'Order Id':'order_id', 'Order Date':'order_date'})
#df.columns=df.columns.str.lower()
#df.columns=df.columns.str.replace(' ','_')
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [53]:
#derive new columns discount , sale price and profit

#df['discount'] = df['list_price']*df['discount_percent']/100
#df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price']-df['cost_price']

In [61]:
#convert order date from object data type to datetime

df['order_date'] = pd.to_datetime(df['order_date'],format='%y-%m-%d')


order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [64]:
#drop cost price list price and discount percent columns

df.drop(columns=['cost_price','list_price','discount_percent'], inplace=True)

In [ ]:
#load the data into sql server using replace option

import sqlalchemy as sal

# Step 1: Create engine for your SQL Server

engine = sal.create_engine(
    "mssql+pyodbc://BOOK-TT4RT3CNBE\MSSQLSERVER08/PROJECTS?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

# Step 2: Connect to the database
conn = engine.connect()


df.to_sql('df_orders', con=conn , index=False, if_exists = 'replace')


In [79]:
#load the data into sql server using append option to save memory (as using replace pandas will create highest possible datatypes )
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

38

In [81]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')